In [ ]:
import pandas as pd
import csv
import numpy as np
from tqdm import tqdm

def load_mapping(file_path, input_columns):
    mapping_df = pd.read_csv(file_path, index_col=0)
    mapping_dicts = {}
    for col in tqdm(mapping_df.columns, desc="Loading file"):
        if col in input_columns:
            mapping_dicts[col] = mapping_df[col].dropna().to_dict()
    return mapping_dicts

def should_remove(row, mapping_dicts):
    return any(mapping_dict.get(row[col_name]) == "Unknown" for col_name, mapping_dict in mapping_dicts.items())

def replace_codes(df, mapping_dicts):
    for col_name, mapping_dict in tqdm(mapping_dicts.items(), desc="Replacing integers with strings"):
        if col_name in df.columns:
            df[col_name] = df[col_name].replace(mapping_dict)
    return df

def reverse_mapping(mapping_dicts):
    reverse_dicts = {}
    for col_name, mapping_dict in mapping_dicts.items():
        reverse_dict = {}
        for k, v in mapping_dict.items():
            if v in reverse_dict:
                reverse_dict[v] = min(reverse_dict[v], k)
            else:
                reverse_dict[v] = k
        reverse_dicts[col_name] = reverse_dict
    return reverse_dicts

def replace_strings(df, reverse_dicts):
    for col_name, reverse_dict in tqdm(reverse_dicts.items(), desc="Replacing strings with lowest integers"):
        if col_name in df.columns:
            df[col_name] = df[col_name].replace(reverse_dict)
    return df

def save_dicts_to_csv(reverse_dicts, file_path):
    df = pd.DataFrame(reverse_dicts)
    df.to_csv(file_path, index=False)
    print("Dictionary saved to file")

# Read the input data
input_data_file_path = input("Enter the path to the dataset CSV file: ")
df = pd.read_csv(input_data_file_path)

# Load mapping dictionaries from a CSV file
mapping_file_path = input("Enter the path to the mapping CSV file: ")
mapping_dicts = load_mapping(mapping_file_path, df.columns)

# Filter out rows with "Unknown" values
print("Dataset filtering started")
df = df[~df.apply(lambda row: should_remove(row, mapping_dicts), axis=1)]
print("Dataset filtering completed")

# Replace numerical codes with string descriptions
df = replace_codes(df, mapping_dicts)

# Save string data
file_name = input("Please enter the desired filtered dataset name (e.g., 'dataset'): ")
df.to_csv('str_' + file_name + '.csv', index=False)
print("String dataset file saved")

# Create reverse dictionaries and save them to a CSV file
reverse_dicts = reverse_mapping(mapping_dicts)
dict_file_name = input("Enter the desired name for the dictionary file (e.g., 'dictionary'): ")
save_dicts_to_csv(reverse_dicts, dict_file_name + '.csv')

# Replace strings with the lowest integer values
df = replace_strings(df, reverse_dicts)

# Save integer data
df.to_csv('int_' + file_name + '.csv', index=False)
print("Integer dataset file saved")
